In [1]:
from PIL import Image
import os
import pandas as pd
import numpy as np
import cv2

In [2]:
#!pip install opencv-python

In [3]:
# Load the telemetry data again
lidar_data = pd.read_csv("Data/commercial_lidar.csv")
dlc_data = pd.read_csv("Data/dlc.csv")
truth_data = pd.read_csv("Data/truth.csv")

# Show the first few rows of each DataFrame
lidar_data.head(), dlc_data.head(), truth_data.head()


(   TIME_NANOSECONDS_TAI   OMPS_Range_M[1]   OMPS_Range_M[2]   \
 0           1.602596e+18           22.1875           20.4375   
 1           1.602596e+18           22.1250           20.5625   
 2           1.602596e+18           22.1250           20.3750   
 3           1.602596e+18           22.1250           20.5625   
 4           1.602596e+18           22.1250           20.5625   
 
    OMPS_Range_M[3]   OMPS_Range_M[4]   OMPS_DopplerSpeed_MpS[1]   \
 0           28.0625            21.750                  -0.160156   
 1           28.1250            21.875                  -0.394531   
 2           27.8750            21.750                  -0.187500   
 3           28.0625            21.750                   0.550781   
 4           28.1250            21.625                   0.628906   
 
    OMPS_DopplerSpeed_MpS[2]   OMPS_DopplerSpeed_MpS[3]   \
 0                  -1.292969                   0.347656   
 1                   0.171875                   0.320312   
 2          

In [4]:
# Directory where images are stored
image_directory = "Images"

# List to store preprocessed images
preprocessed_images = []

# Get list of image files
image_files = os.listdir(image_directory)

# Sort the image files
image_files.sort()

for file_name in image_files:
    # Check the file extension
    _, extension = os.path.splitext(file_name)
    if extension.lower() not in ['.png', '.pgm']:
        continue

    # Construct full file path
    file_path = os.path.join(image_directory, file_name)
    
    try:
        # Open the image file
        img = Image.open(file_path)

        # Convert the image to grayscale
        img_gray = img.convert("L")

        # Resize the image (if needed)
        #img_resized = img_gray.resize((width, height))  # replace width and height with desired values

        # Add the preprocessed image to the list
        preprocessed_images.append(img_gray)
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

Error processing file image_1602596283268870144.000000.pgm: buffer is not large enough
Error processing file image_1602596331270569728.000000.pgm: buffer is not large enough
Error processing file image_1602596366270422528.000000.pgm: buffer is not large enough
Error processing file image_1602596452268933120.000000.pgm: buffer is not large enough
Error processing file image_1602596578268905984.000000.pgm: cannot identify image file 'Images\\image_1602596578268905984.000000.pgm'


In [5]:
# parameters for ShiTomasi corner detection
feature_params = dict(maxCorners = 100, qualityLevel = 0.3, minDistance = 7)

# parameters for lucas kanade optical flow
lk_params = dict(winSize = (15,15), maxLevel = 2)

# take first frame and find corners in it
old_frame = np.array(preprocessed_images[0])
p0 = cv2.goodFeaturesToTrack(old_frame, mask = None, **feature_params)

# create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

for i in range(1, len(preprocessed_images)):
    frame = np.array(preprocessed_images[i])
    
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_frame, frame, p0, None, **lk_params)
    
    # if optical flow is found, proceed with the calculations
    if p1 is not None:
        # select good points
        good_new = p1[st==1]
        good_old = p0[st==1]
        
        # draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (int(a),int(b)), (int(c),int(d)), (0, 255, 0), 2)
            frame = cv2.circle(frame, (int(a),int(b)), 5, (0, 0, 255), -1)
        img = cv2.add(frame, mask)
        
        # now update the previous frame and previous points
        old_frame = frame.copy()
        p0 = good_new.reshape(-1, 1, 2)
